<a href="https://colab.research.google.com/github/SupermarketAutomationAI/baseline_model_SVM/blob/main/baseline_model_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the Data - V1

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
! unzip '/content/drive/My Drive/APS360 Project/Dataset Zip Files/SupermarketAutomationAI_Dataset_V1' -d '/root/datasets'

Streaming output truncated to the last 5000 lines.
  inflating: /root/datasets/SupermarketAutomationAI_Dataset_V1/Multiple Oranges/._Orange00784.png  
  inflating: /root/datasets/SupermarketAutomationAI_Dataset_V1/Multiple Oranges/Orange00785.png  
  inflating: /root/datasets/__MACOSX/SupermarketAutomationAI_Dataset_V1/Multiple Oranges/._Orange00785.png  
  inflating: /root/datasets/SupermarketAutomationAI_Dataset_V1/Multiple Oranges/._Orange00785.png  
  inflating: /root/datasets/SupermarketAutomationAI_Dataset_V1/Multiple Oranges/Orange00786.png  
  inflating: /root/datasets/__MACOSX/SupermarketAutomationAI_Dataset_V1/Multiple Oranges/._Orange00786.png  
  inflating: /root/datasets/SupermarketAutomationAI_Dataset_V1/Multiple Oranges/._Orange00786.png  
  inflating: /root/datasets/SupermarketAutomationAI_Dataset_V1/Multiple Oranges/Orange00787.png  
  inflating: /root/datasets/__MACOSX/SupermarketAutomationAI_Dataset_V1/Multiple Oranges/._Orange00787.png  
  inflating: /root/datasets/

Split the data

In [2]:
# Import the necessary libraries
import json
import os
import math
import shutil

In [3]:
# create data directory and move all images into it
parent_dir = "/root/datasets/SupermarketAutomationAI_Dataset_V1"
os.chdir(parent_dir)
category_list = list(filter(lambda x: os.path.isdir(x), os.listdir()))
data_dir = parent_dir + '/' + "data"
os.mkdir(data_dir, 755)
for category in category_list:
    cat_dir = parent_dir + '/' + category
    shutil.move(cat_dir, data_dir)

In [4]:
train_split = 0.6

dataset_dirs= ['train','val','test']
for dsdirs in dataset_dirs:
 path = parent_dir + '/'+ dsdirs
 os.mkdir( path,755 )

for category in category_list: 
    src_path = parent_dir + '/data/' + category
    train_dir = parent_dir + '/train/' + category + '/'
    val_dir = parent_dir + '/val/' + category + '/'
    test_dir = parent_dir + '/test/' + category + '/'
    
    os.mkdir(train_dir, 755 )
    os.mkdir(val_dir, 755)
    os.mkdir(test_dir, 755)

    #get files' names list from respective directories
    os.chdir(src_path)
    files = [f for f in os.listdir() if os.path.isfile(f)]

    #get training, testing and validation files count
    train_count = math.ceil(train_split*len(files))
    valid_count = int((len(files)-train_count)/2)
    test_count = valid_count

    #get files to segragate for train,test and validation data set
    train_data_list = files[0: train_count-1]
    valid_data_list = files[train_count:train_count+valid_count-1] 
    test_data_list = files[train_count+valid_count:]


    for train_data in train_data_list:
        train_path = src_path + '/' + train_data
        shutil.move(train_path,train_dir)

    for valid_data in valid_data_list:
        valid_path = src_path + '/' + valid_data
        shutil.move(valid_path,val_dir)

    for test_data in test_data_list:
        test_path = src_path + '/' + test_data
        shutil.move(test_path,test_dir)

    # Move any files that are left behind into the training directory
    os.chdir(src_path)
    files = [f for f in os.listdir() if os.path.isfile(f)]
    for img_left_behind in files:
        img_path = src_path + '/' + img_left_behind
        shutil.move(img_path, train_dir)
    
    # TODO: Print number of files in directory, and compute the actual split porportions
    os.chdir(src_path)
    files = [f for f in os.listdir() if os.path.isfile(f)]
    src_files = len(files)
    print("SRC - {} : {} files".format(category, src_files))
    os.chdir(train_dir)
    files = [f for f in os.listdir() if os.path.isfile(f)]
    train_files = len(files)
    print("TRAIN - {} : {} files".format(category, train_files))
    os.chdir(val_dir)
    files = [f for f in os.listdir() if os.path.isfile(f)]
    val_files = len(files)
    print("VAL - {} : {} files".format(category, val_files))
    os.chdir(test_dir)
    files = [f for f in os.listdir() if os.path.isfile(f)]
    test_files = len(files)
    print("TEST - {} : {} files".format(category, test_files))

    total_files = train_files + val_files + test_files
    print("Split: {}% training, {}% validation, and {}% test".format((train_files/total_files * 100), (val_files/total_files * 100), (test_files/total_files * 100)))

SRC - Multiple Bananas : 0 files
TRAIN - Multiple Bananas : 3318 files
VAL - Multiple Bananas : 1104 files
TEST - Multiple Bananas : 1106 files
Split: 60.02170767004341% training, 19.971056439942114% validation, and 20.00723589001447% test
SRC - Multiple Red Delicious Apples : 0 files
TRAIN - Multiple Red Delicious Apples : 772 files
VAL - Multiple Red Delicious Apples : 255 files
TEST - Multiple Red Delicious Apples : 257 files
Split: 60.12461059190031% training, 19.859813084112147% validation, and 20.01557632398754% test
SRC - Multiple Gala Apples : 0 files
TRAIN - Multiple Gala Apples : 675 files
VAL - Multiple Gala Apples : 223 files
TEST - Multiple Gala Apples : 224 files
Split: 60.16042780748663% training, 19.875222816399287% validation, and 19.964349376114082% test
SRC - Multiple Granny Smith Apples : 0 files
TRAIN - Multiple Granny Smith Apples : 993 files
VAL - Multiple Granny Smith Apples : 329 files
TEST - Multiple Granny Smith Apples : 330 files
Split: 60.1089588377724% tra